In [6]:
# -*- coding: utf-8 -*-
import os
import numpy as np
import pickle
import time
import csv
class Gas(object):

    def __init__(self, value, index=5):
        self.index = index
        self.judge = False
        self.org = value
        self.value = []
        self.base_rate = 0.8
        self.max_rate = 0
        self.result = self.fd(value)
        self.sem = 1 - self.max_rate
        if self.judge:
            self.median = self.fit()
            self.last = self.week(index)

    #预算
    def week(self, index):
        one = np.array(self.org[-3*index:-2*index])
        two = np.array(self.org[-2*index:-index])
        three = np.array(self.org[-index:])
        me = np.array([self.median]*index)
        print one,two,three,me
        return (((one-me)*0.23+(two-me)*0.27+(three-me)*0.31)/3 + me).tolist()

    # 获取中位数以及中位数所在高斯分布列表
    def get_mid_digit(self, list, step):
        list.sort()
        mid = list[len(list) / 2]
        value_list = []
        for i in range(1, 6):
            value = mid - mid*(i * step)
            value_list.append(int(value))
        value_list.reverse()
        for i in range(1, 6):
            value = mid + mid*(i*step)
            value_list.append(int(value))
        return value_list, mid

    def distribute_rate(self, value, dis):    # value代表list， dis代表中位数所在高斯分布列表
        rate_count = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]  # 各个分段的 count
        for v in value:
            if v < dis[0]:
                rate_count[0] += 1
            elif dis[0] <= v < dis[1]:
                rate_count[1] += 1
            elif dis[1] <= v < dis[2]:
                rate_count[2] += 1
            elif dis[2] <= v < dis[3]:
                rate_count[3] += 1
            elif dis[3] <= v < dis[4]:
                rate_count[4] += 1
            elif dis[4] <= v < dis[5]:
                rate_count[5] += 1
            elif dis[5] <= v < dis[6]:
                rate_count[6] += 1
            elif dis[6] <= v < dis[7]:
                rate_count[7] += 1
            elif dis[7] <= v < dis[8]:
                rate_count[8] += 1
            elif dis[8] <= v < dis[9]:
                rate_count[9] += 1
            elif dis[9] <= v:
                rate_count[10] += 1
        rate = [round(r / float(len(value)), 2) for r in rate_count] # count 转化为比例
        return rate

    # 过滤数据
    def fd(self, value):
        self.max_rate = 1 - self.base_rate
        # 循环获取最佳分布数列
        if len(value) >= 10:
            self.value = value
            tmp = False
            for d in range(0, len(value), 1):
                dis, mid = self.get_mid_digit(value[d:], self.max_rate)
                f_data = self.distribute_rate(value[d:], dis)
                if sum(f_data[3:8]) > self.base_rate and len(value[d:]) >= len(value)/2:
                    self.value = value[d:]
                    self.base_rate += 0.01
                    self.max_rate -= 0.01
                    self.judge = True
                    tmp = True
                elif 2 <= len(value[d:]) < len(value)/2 or tmp:
                    break
                else:
                    pass
        else:
            return self.judge
        if len(self.value) <= 1:
            self.judge = False
            return self.judge
        elif self.judge:
            self.mid = mid
            self.f_data = f_data
            self.dis = dis
            return self.value, mid, f_data, dis, self.base_rate, self.max_rate
        else:
            return self.judge

    def fit(self):
        for n, i in enumerate(self.f_data):
            if i == max(self.f_data):
                return (self.dis[4]+self.dis[5])/2


def open_pkl(files):
    for f in files:
        try:
            output = open(f, 'rb')
            data = pickle.load(output)
        except Exception as e:
            print (f, "duce error")
        yield data


def work_free_day(date):
    def get_day(time):
        tm = {}
        for d in time:
            t = d[2].strftime("%Y%m%d %H:%M:%S")[:8]
            count = tm.get(t, None)
            tm[t] = tm[t] + 1 if count != None else 1
        return tm, d[1]
    tm, shop_id = get_day(date)
    timestamp = tm.keys()
    timestamp.sort()
    day = [int(l) for l in timestamp]
    work = []
    free = []
    for ind in day:
        oth = [20160929, 20160930, 20161001, 20161002, 20161003, 20161004, 20161005,
               20161006, 20161007, 20161008, 20161009, 20161010, 20161011, 20161012]
        if ind in oth:
            pass
        else:
            timeArray = time.strptime(str(ind), "%Y%m%d")
            timeStamp = int(time.mktime(timeArray))
            timeArray = time.localtime(timeStamp)
            ju = time.strftime("%A", timeArray).upper()
            mon = 'MONDAY'
            tues = 'TUESDAY'
            wed = 'WEDNESDAY'
            thurs = 'THURSDAY'
            fri = 'FRIDAY'
            satur = 'SATURDAY'
            sun = 'SUNDAY'
            if ju == mon:
                work.append(tm[str(ind)])
            elif ju == tues:
                work.append(tm[str(ind)])
            elif ju == wed:
                work.append(tm[str(ind)])
            elif ju == thurs:
                work.append(tm[str(ind)])
            elif ju == fri:
                work.append(tm[str(ind)])
            elif ju == satur:
                free.append(tm[str(ind)])
            elif ju == sun:
                free.append(tm[str(ind)])
            else:
                pass
    return work, free, shop_id
"""
if __name__ == '__main__':
    files = [r'C:\project\ShopEvaluation\ali\db\1.pkl', ]
    # list = os.listdir(r'C:\project\ShopEvaluation\ali\db')
    # files = []
    # for line in list:
    #     filepath = os.path.join(r'C:\project\ShopEvaluation\ali\db', line)
    #     files.append(filepath)
    data = open_pkl(files=files)
    for n, d in enumerate(data):
        result_last = []
        ls = work_free_day(d)
        pre = []
        for d, test in enumerate(ls):
            if d == 0:
                a = Gas(test[-15:], 5)
                if a.judge:
                    pre += a.last
                else:
                    pass
            elif d == 1:
                a = Gas(test[-12:], 2)
                if a.judge:
                    pre += a.last
                else:
                    pass
            elif d == 2:
                if len(pre) >= 6:
                    pre = [pre[0], pre[1], pre[2], pre[3], pre[5], pre[6], pre[4]]*2
                    shop_id = test
                    pre.insert(0, int(shop_id))
                else:
                    shop_id = test
                    pre.insert(0, int(shop_id))
            else:
                pass
        if len(pre) == 15:
            result_last = pre
            csvfile = file('result0224-1.csv', 'ab+')
            writer = csv.writer(csvfile)
            writer.writerow(result_last)
            csvfile.close()
        else:
            print pre[0]
"""

"\nif __name__ == '__main__':\n    files = [r'C:\\project\\ShopEvaluation\x07li\\db\x01.pkl', ]\n    # list = os.listdir(r'C:\\project\\ShopEvaluation\x07li\\db')\n    # files = []\n    # for line in list:\n    #     filepath = os.path.join(r'C:\\project\\ShopEvaluation\x07li\\db', line)\n    #     files.append(filepath)\n    data = open_pkl(files=files)\n    for n, d in enumerate(data):\n        result_last = []\n        ls = work_free_day(d)\n        pre = []\n        for d, test in enumerate(ls):\n            if d == 0:\n                a = Gas(test[-15:], 5)\n                if a.judge:\n                    pre += a.last\n                else:\n                    pass\n            elif d == 1:\n                a = Gas(test[-12:], 2)\n                if a.judge:\n                    pre += a.last\n                else:\n                    pass\n            elif d == 2:\n                if len(pre) >= 6:\n                    pre = [pre[0], pre[1], pre[2], pre[3], pre[5], pre[6], pr

In [18]:
test = [12,2,4,1,7,8,3,11,6,7,2,56,88,21,0]
print test

[12, 2, 4, 1, 7, 8, 3, 11, 6, 7, 2, 56, 88, 21, 0]


In [19]:
a = Gas(test[-15:], 5)

In [20]:
#self.value, mid, f_data, dis, self.base_rate, self.max_rate
print a.result

False
